# Face Attendence(Hough Transformation)

In [1]:
import cv2
import face_recognition
import numpy as np
import os
from datetime import datetime

In [2]:
path='images'

In [3]:
images=[]
personName=[]
encodeList=[]
myList=os.listdir(path)
for i in myList:
    ci=cv2.imread(f'{path}/{i}')
    images.append(ci)
    personName.append(os.path.splitext(i)[0])

In [12]:
#face encoding(using Hough algorith)
def faceEncoding(images):
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList
encodeListKnown=encodeList

In [5]:
def attendance(name):
    with open('Attend.cvs', 'r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            time_now = datetime.now()
            tStr = time_now.strftime('%H:%M:%S')
            dStr = time_now.strftime('%d/%m/%Y')
            f.writelines(f'\n{name},{tStr},{dStr}')

In [ ]:
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    faces=cv2.resize(frame,(0,0),None,0.25,0.25)
    face=cv2.cvtColor(faces,cv2.COLOR_BGR2RGB)
    facesCurrentFrame=face_recognition.face_locations(faces)
    encodeCurrentFrame=face_recognition.face_encodings(faces,facesCurrentFrame)
    for encodeFace,faceLoc in zip(encodeCurrentFrame,facesCurrentFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
        face_distance=face_recognition.face_distance(encodeListKnown,encodeFace)
        
        matchIndex=np.argmin(face_distance)
        if matches[matchIndex]:
            name=personName[matchIndex].upper()
            print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(frame,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            attendance(name)
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1)==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [4]:
print(personName)

['bill gates', 'donald trump', 'elon musk', 'Geeta Devi', 'jeff bezos', 'obama']
